# Code to reproduce the figures of Day et al., *The Cryosphere*, 2025

Prerequisites:
- 

In [3]:
from io import BytesIO
import xarray as xr
import matplotlib
import time
# import imageio.v3 as iio
import glob
from datetime import datetime, timedelta
import os
from PIL import Image
import cartopy.crs as ccrs
import matplotlib.pyplot as plt

# %matplotlib inline
from matplotlib.colors import LogNorm
from matplotlib.colors import SymLogNorm
from matplotlib.colors import Normalize
from matplotlib import gridspec
import matplotlib.colors as mcolors
from matplotlib.patches import Circle

import seaborn as sns
import geopy.distance
import textwrap
import metpy.calc as mpcalc
from metpy.units import units
from geopy.distance import geodesic
import platform
import sys
sys.path.append('/home/566/nd0349/antarctic-cyclones/functions')
from system_setup import *
savepath, netcdfpath = getSavePath()
from breakup_indentification import *
from formatting import *# convertFilenameToDatetime, getCycloneDatetime, getLongtiudeIndices, fixCiceDatetimes, subdomainCiceAndJra55
from plotting import *#create_map_axis, plot_style, add_subplot_label, set_ax_date, plotSectorMap, plotSectorMap, addGridLabels
from fstd import *#ciceBinWidths, integrateFloeSize
from simple_maths import *#getClosestPoint1D, getClosestPoint2D, timeDerivative

def fixCyclicPoint(data, datatype='jra'):
    if datatype=='jra':
        if abs(np.diff(data.longitude, 1)).max() > 180:
            temp_lons = data.longitude.apply(lambda x: x - 360 if x > 180 else x)
            data.longitude = temp_lons
        else:
            temp_lons = data.longitude
    elif datatype=='CICE_lon':
        if abs(abs(np.diff(data.lon.values,1))).max() > 180:
            temp_lons = data.lon.values
            idx = temp_lons > 180
            temp_lons[idx] = temp_lons[idx] - 360
        else:
            temp_lons = data.lon.values
    elif datatype=='CICE':
        if abs(abs(np.diff(data.TLON.values,1))).max() > 180:
            temp_lons = data.TLON.values
            idx = temp_lons > 180
            temp_lons[idx] = temp_lons[idx] - 360
        else:
            temp_lons = data.TLON.values
    else:
        if abs(abs(np.diff(data,1))).max() > 180:
            temp_lons = data
            idx = temp_lons > 180
            temp_lons[idx] = temp_lons[idx] - 360
        else:
            temp_lons = data

    return temp_lons

def getEachBreakupIdx(breakup_idx):
    '''
    Given an boolean array of when the breakup threshold has been met return each breakup event. 
    A breakup event is defined as when continued breakup occurs.

    Input: Boolean array
    Output: Stacked boolean array of each breakup event
    '''

    int_arr = breakup_idx.astype(int)
    # Add an artificial value from the start
    int_arr = np.insert(int_arr, 0, 0)
    diff = np.diff(int_arr)
    transition_indexes = np.where(diff == 1)[0]
    transition_indexes_end = np.where(diff == -1)[0] + 1
    
    # Add a starting point if the first element in True
    # if transition_indexes[0]:
    #     transition_indexes = np.insert(transition_indexes, 0, 0)
    
    breakup_idx_stacked = []
    
    # Stack these breakup events
    for i in np.arange(0, len(transition_indexes), 1):
        new_array = np.full(breakup_idx.shape, False)
        if i >= len(transition_indexes_end):
            new_array[transition_indexes[i]:] = breakup_idx[transition_indexes[i]:]
            # print( breakup_idx[transition_indexes[i]])
        else:
            new_array[transition_indexes[i]:transition_indexes_end[i]] = breakup_idx[transition_indexes[i]:transition_indexes_end[i]]
            # print(breakup_idx[transition_indexes[i]:transition_indexes_end[i]+1])
        breakup_idx_stacked.append(new_array)
    
    # Check to see if there are any errors
    if ~(np.sum(breakup_idx_stacked, axis=0).astype(bool) == breakup_idx).all():
        print("Error: Breakup indexes not retained!")
        print(np.sum(breakup_idx_stacked, axis=0).astype(bool))
        print(breakup_idx)

    return breakup_idx_stacked

def getMajorEvents(breakup_events_array, time_threshold=6):
    # Given a stacked array of all breakup events return those that persist for longer than time_threshold:
    idx_bool = np.sum(breakup_events_array, axis=1) >= time_threshold
    idx = np.where(idx_bool)[0]

    major_events_stacked = []
    for i in idx:
        major_events_stacked.append(breakup_events_array[i])
    return major_events_stacked

# import scienceplots
plt.style.use('default')
import matplotlib.colors as mcolors
# plt.style.use(['nature', 'grid'])
textwidth = 3.31314*2
aspect_ratio = 6/8
scale = 1.0
width = textwidth * scale
height = width * aspect_ratio/2

diff = 1121 # Difference in the CICE grid from 0 degrees E


fontsize = 12
ticks_fontsize = 10
label_x = 0.025
label_y = 0.975

ModuleNotFoundError: No module named 'scienceplots'

In [ ]:
# Plot settings
import scienceplots
textwidth = 3.31314*2
aspect_ratio = 6/8
scale = 1.0
width = textwidth * scale
height = width * aspect_ratio
plt.style.use(['science'])

plt.rcParams.update({
    "text.usetex": True,
    "font.family": "sans-serif",
    # "font.sans-serif": "Helvetica",
})

plt.rcParams['axes.titlesize'] = 14     # Font size for axes titles
plt.rcParams['axes.labelsize'] = 12     # Font size for x and y labels
plt.rcParams['xtick.labelsize'] = 10    # Font size for x tick labels
plt.rcParams['ytick.labelsize'] = 10    # Font size for y tick labels
plt.rcParams['legend.fontsize'] = 12    # Font size for the legend
plt.rcParams['figure.titlesize'] = 16   # Font size for figure title

from matplotlib.ticker import ScalarFormatter

# Assuming you already have the colorbars (cbar1, cbar2, cbar3)
formatter = ScalarFormatter(useMathText=True)
formatter.set_scientific(True)
formatter.set_powerlimits((-2, 2))  # Display scientific notation for values outside this range

# Plot settings
import scienceplots
plt.style.use('default')
import matplotlib.colors as mcolors
plt.style.use(['nature', 'grid'])
textwidth = 3.31314*2
aspect_ratio = 6/8
scale = 1.0
width = textwidth * scale
height = width * aspect_ratio/2

import scienceplots
textwidth = 3.31314*2
aspect_ratio = 6/8
scale = 1.0
width = textwidth * scale
height = width * aspect_ratio
plt.style.use(['science'])

plt.rcParams.update({
    "text.usetex": True,
    "font.family": "sans-serif",
    # "font.sans-serif": "Helvetica",
})

plt.rcParams['axes.titlesize'] = 14     # Font size for axes titles
plt.rcParams['axes.labelsize'] = 12     # Font size for x and y labels
plt.rcParams['xtick.labelsize'] = 10    # Font size for x tick labels
plt.rcParams['ytick.labelsize'] = 10    # Font size for y tick labels
plt.rcParams['legend.fontsize'] = 12    # Font size for the legend
plt.rcParams['figure.titlesize'] = 16   # Font size for figure title

from matplotlib.ticker import ScalarFormatter

# Assuming you already have the colorbars (cbar1, cbar2, cbar3)
formatter = ScalarFormatter(useMathText=True)
formatter.set_scientific(True)
formatter.set_powerlimits((-2, 2))  # Display scientific notation for values outside this range

testPlot()